In [1]:
import pandas

data = pandas.read_csv('Python(view_10000).csv', sep=',', error_bad_lines=False)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
data.head

<bound method NDFrame.head of              Id  PostTypeId  AcceptedAnswerId  ParentId         CreationDate  \
0           502           1            7090.0       NaN  2008-08-02 17:01:58   
1           594           1             595.0       NaN  2008-08-03 01:15:08   
2           742           1           33957.0       NaN  2008-08-03 15:55:28   
3           773           1            7286.0       NaN  2008-08-03 18:27:09   
4           972           1            2982.0       NaN  2008-08-04 02:17:51   
5          1171           1           28705.0       NaN  2008-08-04 12:00:57   
6          1476           1           13107.0       NaN  2008-08-04 18:20:36   
7          1829           1            1852.0       NaN  2008-08-05 02:39:23   
8          1854           1            1857.0       NaN  2008-08-05 03:23:18   
9          2933           1            2937.0       NaN  2008-08-05 22:26:00   
10         3061           1            3071.0       NaN  2008-08-06 03:36:08   
11        

In [3]:
#java -mx5g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -timeout 10000
from politeness.helpers import set_corenlp_url
from politeness.classifier import Classifier

set_corenlp_url("http://localhost:9000")
cls = Classifier()

In [3]:
data.shape[0]

36087

In [4]:
from datetime import datetime
import csv
import nltk
import re
from bs4 import BeautifulSoup
from time import sleep

#Function to clean the html
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

outFile = open("Python(view_10000)_ANALYZED.csv", 'w', newline='')
fieldnames = ["Id", "PostTypeId", "AcceptedAnswerId", "ParentId", "CreationDate", "DeletionDate", "Score", "ViewCount", "Body", "BodyNOHTML", "PolitenessConfidence", "ImpolitenessConfidence", "OwnerUserId", "OwnerDisplayName", "LastEditorUserId", "LastEditorDisplayName", "LastEditDate", "LastActivityDate", "Title", "Tags","AnswerCount", "CommentCount", "FavoriteCount", "ClosedDate", "CommunityOwnedDate"]
w = csv.DictWriter(outFile, fieldnames = fieldnames)
w.writeheader() 

skips = 0

for i in range(data.shape[0]):
    try:
        tempDict = {}
        
        tempDict["Id"] = str(data.iloc[i, 0])
        tempDict["PostTypeId"] = str(data.iloc[i, 1])
        tempDict["AcceptedAnswerId"] = str(data.iloc[i, 2])
        tempDict["ParentId"] = str(data.iloc[i, 3])
        tempDict["CreationDate"] = str(data.iloc[i, 4])
        tempDict["DeletionDate"] = str(data.iloc[i, 5])
        tempDict["Score"] = str(data.iloc[i, 6])
        tempDict["ViewCount"] = str(data.iloc[i, 7])
        tempDict["Body"] = str(data.iloc[i, 8])
        
        #Remove html and code snippets
        temp = str(data.iloc[i, 8])
        soup = BeautifulSoup(str(data.iloc[i, 8]), "lxml")
        removals = soup.find_all('code')
        for match in removals:
            match.decompose()
        text = soup.get_text()
        text = text.replace('\n', ' ').replace('\r', '').replace('\t', '') #remove newlines and such
        
        #cleaned = cleanhtml(data.iloc[i, 8])
        
        #cleaned = cleaned.replace('\n', ' ')
        output = cls.predict(text)
        vals = list(output[-1].values()) #The last entry in the analyzed array of values are the scores for the entire doc
        
        tempDict["BodyNOHTML"] = text
        
        tempDict["PolitenessConfidence"] = str(vals[0][0])
        tempDict["ImpolitenessConfidence"] = str(vals[0][1])
        tempDict["OwnerUserId"] = str(data.iloc[i, 9])
        tempDict["OwnerDisplayName"] = str(data.iloc[i,10])
        tempDict["LastEditorUserId"] = str(data.iloc[i, 11])
        tempDict["LastEditorDisplayName"] = str(data.iloc[i, 12])
        tempDict["LastEditDate"] = str(data.iloc[i, 13])
        tempDict["LastActivityDate"] = str(data.iloc[i, 14])
        tempDict["Title"] = str(data.iloc[i, 15])
        tempDict["Tags"] = str(data.iloc[i, 16])
        tempDict["AnswerCount"] = str(data.iloc[i, 17])
        tempDict["CommentCount"] = str(data.iloc[i, 18])
        tempDict["FavoriteCount"] = str(data.iloc[i, 19])
        tempDict["ClosedDate"] = str(data.iloc[i, 20])
        tempDict["CommunityOwnedDate"] = str(data.iloc[i, 21])
        
        if (i % 1000 == 0): 
            print("entry: " + str(i) + " of " + str(data.shape[0]))
        w.writerow(tempDict)
    except:
        print("Total skips = " + str(skips)) #keep track of how many strings were skipped 
        skips += 1
        continue

print(skips)
outFile.close()

entry: 0 of 36087
entry: 1000 of 36087
entry: 2000 of 36087
entry: 3000 of 36087


/home/austin/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/austin/.local/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


entry: 4000 of 36087
entry: 5000 of 36087
entry: 6000 of 36087
entry: 7000 of 36087


Exception
  Sentence: _mysql.c:364: warning: initialization from incompa
Traceback (most recent call last):
  File "/home/austin/.local/lib/python3.6/site-packages/politeness/helpers.py", line 130, in get_parses
    response.raise_for_status()
  File "/home/austin/.local/lib/python3.6/site-packages/requests/models.py", line 940, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 500 Server Error: Internal Server Error for url: http://localhost:9000/?properties=%7B%27annotators%27%3A+%27tokenize%2Cssplit%2Cpos%2Cparse%2Cdepparse%27%7D


Total skips = 0
entry: 8000 of 36087


...

Total skips = 7
entry: 30000 of 36087
entry: 31000 of 36087


Exception
  Sentence: here's the dir() of rotated_image: ['_Image__trans
Traceback (most recent call last):
  File "/home/austin/.local/lib/python3.6/site-packages/politeness/helpers.py", line 130, in get_parses
    response.raise_for_status()
  File "/home/austin/.local/lib/python3.6/site-packages/requests/models.py", line 940, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 500 Server Error: Internal Server Error for url: http://localhost:9000/?properties=%7B%27annotators%27%3A+%27tokenize%2Cssplit%2Cpos%2Cparse%2Cdepparse%27%7D


Total skips = 8
entry: 32000 of 36087
entry: 33000 of 36087
entry: 34000 of 36087
entry: 35000 of 36087
entry: 36000 of 36087


Exception
  Sentence: The full error message is the following (seems to 
Traceback (most recent call last):
  File "/home/austin/.local/lib/python3.6/site-packages/politeness/helpers.py", line 130, in get_parses
    response.raise_for_status()
  File "/home/austin/.local/lib/python3.6/site-packages/requests/models.py", line 940, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 500 Server Error: Internal Server Error for url: http://localhost:9000/?properties=%7B%27annotators%27%3A+%27tokenize%2Cssplit%2Cpos%2Cparse%2Cdepparse%27%7D


Total skips = 9
10


In [4]:
from bs4 import BeautifulSoup

#for i in range(data.shape[0]):
for i in range(5):
    temp = str(data.iloc[i, 8])
    print(i)
    print(str(data.iloc[i, 8]))
    soup = BeautifulSoup(str(data.iloc[i, 8]), "lxml")
    removals = soup.find_all('code')
    for match in removals:
        match.decompose()
    text = soup.get_text()
    text = text.replace('\n', ' ').replace('\r', '').replace('\t', '')
    print(str(text))


0
<p>I have a cross-platform (Python) application which needs to generate a JPEG preview of the first page of a PDF.</p>

<p>On the Mac I am spawning <a href="http://developer.apple.com/documentation/Darwin/Reference/ManPages/man1/sips.1.html" rel="noreferrer">sips</a>.  Is there something similarly simple I can do on Windows?</p>

I have a cross-platform (Python) application which needs to generate a JPEG preview of the first page of a PDF. On the Mac I am spawning sips.  Is there something similarly simple I can do on Windows? 
1
<p>There are several ways to iterate over a result set. What are the tradeoff of each?</p>

There are several ways to iterate over a result set. What are the tradeoff of each? 
2
<p><a href="http://www.djangoproject.com/" rel="noreferrer">Django</a> view points to a function, which can be a problem if you want to change only a bit of functionality. Yes, I could have million keyword arguments and even more if statements in the function, but I was thinking mor

In [2]:
import pandas

pythonData = pandas.read_csv("Python(view_10000)_ANALYZED.csv", sep=',', error_bad_lines=False)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
pythonData.head

<bound method NDFrame.head of              Id  PostTypeId  AcceptedAnswerId  ParentId         CreationDate  \
0           502           1            7090.0       NaN  2008-08-02 17:01:58   
1           594           1             595.0       NaN  2008-08-03 01:15:08   
2           742           1           33957.0       NaN  2008-08-03 15:55:28   
3           773           1            7286.0       NaN  2008-08-03 18:27:09   
4           972           1            2982.0       NaN  2008-08-04 02:17:51   
5          1171           1           28705.0       NaN  2008-08-04 12:00:57   
6          1476           1           13107.0       NaN  2008-08-04 18:20:36   
7          1829           1            1852.0       NaN  2008-08-05 02:39:23   
8          1854           1            1857.0       NaN  2008-08-05 03:23:18   
9          2933           1            2937.0       NaN  2008-08-05 22:26:00   
10         3061           1            3071.0       NaN  2008-08-06 03:36:08   
11        

In [9]:
pythonData = pythonData.sort_values(by='PolitenessConfidence', ascending=False)

In [11]:
pythonData.head

<bound method NDFrame.head of              Id  PostTypeId  AcceptedAnswerId  ParentId         CreationDate  \
26338  21752399           1        21755752.0       NaN  2014-02-13 11:06:20   
20856  15799162           1        15799355.0       NaN  2013-04-03 22:12:51   
4598    2399835           1         2399932.0       NaN  2010-03-08 07:18:36   
26310  21702342           1        21711869.0       NaN  2014-02-11 12:49:58   
22915  17872094           1        17872195.0       NaN  2013-07-26 02:51:47   
15879  11070527           1        11070638.0       NaN  2012-06-17 10:10:50   
16702  11774925           1               NaN       NaN  2012-08-02 09:45:24   
29296  26850355           1               NaN       NaN  2014-11-10 18:22:18   
31693  31845258           1        31860747.0       NaN  2015-08-06 01:12:59   
4910    2564137           1         2564161.0       NaN  2010-04-01 22:44:52   
27179  23047296           1        24014635.0       NaN  2014-04-13 19:19:22   
7128    39

In [13]:
print(pythonData.iloc[0:20, :])

             Id  PostTypeId  AcceptedAnswerId  ParentId         CreationDate  \
26338  21752399           1        21755752.0       NaN  2014-02-13 11:06:20   
20856  15799162           1        15799355.0       NaN  2013-04-03 22:12:51   
4598    2399835           1         2399932.0       NaN  2010-03-08 07:18:36   
26310  21702342           1        21711869.0       NaN  2014-02-11 12:49:58   
22915  17872094           1        17872195.0       NaN  2013-07-26 02:51:47   
15879  11070527           1        11070638.0       NaN  2012-06-17 10:10:50   
16702  11774925           1               NaN       NaN  2012-08-02 09:45:24   
29296  26850355           1               NaN       NaN  2014-11-10 18:22:18   
31693  31845258           1        31860747.0       NaN  2015-08-06 01:12:59   
4910    2564137           1         2564161.0       NaN  2010-04-01 22:44:52   
27179  23047296           1        24014635.0       NaN  2014-04-13 19:19:22   
7128    3944655           1         3944

In [20]:
temp = pythonData.iloc[0:20, :]
bottom = pythonData.iloc[-38:-18, :]
temp = temp.append(bottom, ignore_index=True)
temp.head

<bound method NDFrame.head of           Id  PostTypeId  AcceptedAnswerId  ParentId         CreationDate  \
0   21752399           1        21755752.0       NaN  2014-02-13 11:06:20   
1   15799162           1        15799355.0       NaN  2013-04-03 22:12:51   
2    2399835           1         2399932.0       NaN  2010-03-08 07:18:36   
3   21702342           1        21711869.0       NaN  2014-02-11 12:49:58   
4   17872094           1        17872195.0       NaN  2013-07-26 02:51:47   
5   11070527           1        11070638.0       NaN  2012-06-17 10:10:50   
6   11774925           1               NaN       NaN  2012-08-02 09:45:24   
7   26850355           1               NaN       NaN  2014-11-10 18:22:18   
8   31845258           1        31860747.0       NaN  2015-08-06 01:12:59   
9    2564137           1         2564161.0       NaN  2010-04-01 22:44:52   
10  23047296           1        24014635.0       NaN  2014-04-13 19:19:22   
11   3944655           1         3944668.0    

In [22]:
temp = temp.sample(frac=1).reset_index(drop=True)
temp.head
temp.to_csv("Python(view_10000)_TEST.csv")

In [23]:
temp.head

<bound method NDFrame.head of           Id  PostTypeId  AcceptedAnswerId  ParentId         CreationDate  \
0   10866985           1        10866998.0       NaN  2012-06-03 00:25:11   
1   21702342           1        21711869.0       NaN  2014-02-11 12:49:58   
2   17322668           1        17322707.0       NaN  2013-06-26 14:17:23   
3   27760831           1        27760854.0       NaN  2015-01-03 23:47:24   
4   19261833           1        19262349.0       NaN  2013-10-09 02:07:46   
5   16819222           1        16819250.0       NaN  2013-05-29 16:24:16   
6   21752399           1        21755752.0       NaN  2014-02-13 11:06:20   
7   12109648           1               NaN       NaN  2012-08-24 12:45:04   
8   31845258           1        31860747.0       NaN  2015-08-06 01:12:59   
9   34256250           1        34256488.0       NaN  2015-12-13 20:57:50   
10  33417991           1        33420074.0       NaN  2015-10-29 15:11:49   
11   3154624           1         3154717.0    